<a href="https://colab.research.google.com/github/rustbas/MachineLearningHW/blob/main/Practice2/Basyrov_Practice_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [140]:
import torch 
import numpy as np
from sklearn.metrics import roc_auc_score

# Практикум 3, DL for images



## 1. Пример работы со свертками 


In [2]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision import transforms

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))])

training_data = datasets.CIFAR10(
    root='data',
    train=True, 
    download=True,
    transform=transform
)

test_data = datasets.CIFAR10(
    root='data',
    train=False, 
    download=True,
    transform=transform
)

trainloader = torch.utils.data.DataLoader(training_data, batch_size=128, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=128, shuffle=False)


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting data/cifar-10-python.tar.gz to data
Files already downloaded and verified


In [ ]:
from torch import nn
import torch.nn.functional as F

class CNN(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(3, 6, 5)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(6, 16, 5)
    self.fc1 = nn.Linear(16*5*5, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = torch.flatten(x, 1)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x


In [ ]:
import torch.optim as optim

model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.001, momentum = 0.9)

In [ ]:
for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 100 == 0:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,     1] loss: 0.001
[1,   101] loss: 0.115
[1,   201] loss: 0.115
[1,   301] loss: 0.115
[2,     1] loss: 0.001
[2,   101] loss: 0.115
[2,   201] loss: 0.115
[2,   301] loss: 0.115
[3,     1] loss: 0.001


In [ ]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
model.eval()
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = model(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 44 %


In [ ]:
# prepare to count predictions for each class
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print("Accuracy for class {:5s} is: {:.1f} %".format(classname,
                                                   accuracy))

Accuracy for class plane is: 42.3 %
Accuracy for class car   is: 63.9 %
Accuracy for class bird  is: 29.6 %
Accuracy for class cat   is: 26.4 %
Accuracy for class deer  is: 19.4 %
Accuracy for class dog   is: 46.2 %
Accuracy for class frog  is: 56.6 %
Accuracy for class horse is: 56.6 %
Accuracy for class ship  is: 55.9 %
Accuracy for class truck is: 49.0 %


# Задание 1

Написать и обучить нейронную сеть на датасете CIFAR (5 баллов)
1. Замените сверточные слои размера 5х5 на два идущих подряд слоя размером 3х3
2. Обучите модель на GPU, 20 эпох


Дополнительно:
1. (5 баллов) Переписать код с использованием pytorch lightning (см. документацию фреймворка https://pytorch-lightning.readthedocs.io/en/latest/starter/introduction.html)

## 1.1 Два слоя 3х3

In [96]:
from torch import nn
import torch.nn.functional as F

class CNN(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(3, 6, 3)
    self.conv2 = nn.Conv2d(6, 6, 3)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv3 = nn.Conv2d(6, 6, 3)
    self.conv4 = nn.Conv2d(6, 16, 3)
    self.fc1 = nn.Linear(16*5*5, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

  def forward(self, x):
    x = self.conv2(self.conv1(x))
    # print('1 passed', x.shape)
    x = self.pool(F.relu(x))
    # print('2 passed', x.shape)
    x = self.conv4(self.conv3(x))
    # print('3 passed', x.shape)
    x = self.pool(F.relu(x))
    # print('4 passed', x.shape)
    x = torch.flatten(x, 1)
    # print('5 passed', x.shape)
    x = F.relu(self.fc1(x))
    # print('6 passed', x.shape)
    x = F.relu(self.fc2(x))
    # print('7 passed', x.shape)
    x = self.fc3(x)
    # print('8 passed', x.shape)
 
    # x = self.pool(F.relu(self.conv1(x)))
    # x = self.pool(F.relu(self.conv2(x)))
    # x = torch.flatten(x, 1)
    # x = F.relu(self.fc1(x))
    # x = F.relu(self.fc2(x))
    # x = self.fc3(x)

    return x

In [97]:
import torch.optim as optim

model = CNN()
cuda0 = torch.device('cuda')
model.to(cuda0)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.001, momentum = 0.9)

## 1.2 Обучение на 20 эпохах

In [108]:
n_epochs = 20

for epoch in range(n_epochs):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(cuda0), data[1].to(cuda0)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        # print(outputs.device, inputs.device)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        # if i % 100 == 0:    # print every 2000 mini-batches
        #     print(f'[{epoch+1:3d}, {i+1:5d}] loss: {running_loss / 2000:.3f}')
        #           # (epoch + 1, i + 1, running_loss / 2000))
        #     running_loss = 0.0
    print(f'Epoch:{epoch+1:3d}, Running Loss:{running_loss:.3f}')

print('Finished Training')

Epoch:  1, Running Loss:481.190
Epoch:  2, Running Loss:473.614
Epoch:  3, Running Loss:464.994
Epoch:  4, Running Loss:457.919
Epoch:  5, Running Loss:450.200
Epoch:  6, Running Loss:443.650
Epoch:  7, Running Loss:435.540
Epoch:  8, Running Loss:429.851
Epoch:  9, Running Loss:422.252
Epoch: 10, Running Loss:419.163
Epoch: 11, Running Loss:412.038
Epoch: 12, Running Loss:406.141
Epoch: 13, Running Loss:401.435
Epoch: 14, Running Loss:394.654
Epoch: 15, Running Loss:389.568
Epoch: 16, Running Loss:384.526
Epoch: 17, Running Loss:378.779
Epoch: 18, Running Loss:374.023
Epoch: 19, Running Loss:371.148
Epoch: 20, Running Loss:363.904
Finished Training


## 1.3 Проверка

In [161]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
model.eval()
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(cuda0), data[1].to(cuda0)
        # calculate outputs by running images through the network
        outputs = model(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Точность нейросети для {total} изображений: {100*correct/total:.3f}')

Точность нейросети для 10000 изображений: 80.550


In [159]:
# prepare to count predictions for each class
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(cuda0), data[1].to(cuda0)
        outputs = model(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Точность нейросети для класса \'{classname:7s}\': {accuracy:.3f}')

Точность нейросети для класса 'plane  ': 86.100
Точность нейросети для класса 'car    ': 92.200
Точность нейросети для класса 'bird   ': 71.100
Точность нейросети для класса 'cat    ': 69.900
Точность нейросети для класса 'deer   ': 74.100
Точность нейросети для класса 'dog    ': 67.700
Точность нейросети для класса 'frog   ': 90.000
Точность нейросети для класса 'horse  ': 79.000
Точность нейросети для класса 'ship   ': 85.400
Точность нейросети для класса 'truck  ': 90.000


# Задание 2 (5 баллов)
1. Обучитите на GPU претренированную модель VGG (минимум 5 эпох)
2. Протестируйте ее на всех данных и на каждом классе отдельно

Дополнительное задание (5 баллов):     

* Взять какой-нибудь специфичный датасет с картинками
* Взять претренированную VGG (или другу модель) и сделать transfer learning на выбранный датасет

## 2.1 Обучение VGG на GPU

In [145]:
import torchvision.models as models

vgg11 = models.vgg11(pretrained=True).to(cuda0)

In [146]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(vgg11.parameters(), lr = 0.001, momentum = 0.9)

In [147]:
n_epochs = 5

for epoch in range(n_epochs):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(cuda0), data[1].to(cuda0)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = vgg11(inputs)
        # print(outputs.device, inputs.device)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        # if i % 100 == 0:    # print every 2000 mini-batches
        #     print(f'[{epoch+1:3d}, {i+1:5d}] loss: {running_loss / 2000:.3f}')
        #           # (epoch + 1, i + 1, running_loss / 2000))
        #     running_loss = 0.0
    print(f'Epoch:{epoch+1:3d}, Running Loss:{running_loss:.3f}')

print('Finished Training')

Epoch:  1, Running Loss:608.136
Epoch:  2, Running Loss:297.171
Epoch:  3, Running Loss:233.240
Epoch:  4, Running Loss:195.237
Epoch:  5, Running Loss:161.304
Finished Training


## 2.2 Проверка

### 2.2.1 Проверка на всех классах

In [155]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
vgg11.eval()
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(cuda0), data[1].to(cuda0)
        # calculate outputs by running images through the network
        outputs = vgg11(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Точность VGG11 для {total} изображений: {100*correct/total:.3f}')

Точность VGG11 для 10000 изображений: 82.140


### 2.2.2 Проверка на каждом классе отдельно

In [162]:
# prepare to count predictions for each class
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(cuda0), data[1].to(cuda0)
        outputs = vgg11(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Точность VGG11 для класса \'{classname:7s}\': {accuracy:.3f}')

Точность VGG11 для класса 'plane  ': 82.200
Точность VGG11 для класса 'car    ': 91.800
Точность VGG11 для класса 'bird   ': 73.100
Точность VGG11 для класса 'cat    ': 69.700
Точность VGG11 для класса 'deer   ': 77.700
Точность VGG11 для класса 'dog    ': 76.200
Точность VGG11 для класса 'frog   ': 83.700
Точность VGG11 для класса 'horse  ': 86.600
Точность VGG11 для класса 'ship   ': 90.000
Точность VGG11 для класса 'truck  ': 90.400


### 2.2.3 Датасет - CIFAR100, модель - ResNet18

#### Данные

In [168]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))])

training_data = datasets.CIFAR100(
    root='data',
    train=True, 
    download=True,
    transform=transform
)

test_data = datasets.CIFAR100(
    root='data',
    train=False, 
    download=True,
    transform=transform
)

trainloader = torch.utils.data.DataLoader(training_data, batch_size=128, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=128, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [184]:
classes2 = [
'beaver', 'dolphin', 'otter', 'seal', 'whale',
'aquarium fish', 'flatfish', 'ray', 'shark', 'trout',
'orchids', 'poppies', 'roses', 'sunflowers', 'tulips',
'bottles', 'bowls', 'cans', 'cups', 'plates',
'apples', 'mushrooms', 'oranges', 'pears', 'sweet peppers',
'clock', 'computer keyboard', 'lamp', 'telephone', 'television',
'bed', 'chair', 'couch', 'table', 'wardrobe',
'bee', 'beetle', 'butterfly', 'caterpillar', 'cockroach',
'bear', 'leopard', 'lion', 'tiger', 'wolf',
'bridge', 'castle', 'house', 'road', 'skyscraper',
'cloud', 'forest', 'mountain', 'plain', 'sea',
'camel', 'cattle', 'chimpanzee', 'elephant', 'kangaroo',
'fox', 'porcupine', 'possum', 'raccoon', 'skunk',
'crab', 'lobster', 'snail', 'spider', 'worm',
'baby', 'boy', 'girl', 'man', 'woman',
'crocodile', 'dinosaur', 'lizard', 'snake', 'turtle',
'hamster', 'mouse', 'rabbit', 'shrew', 'squirrel',
'maple', 'oak', 'palm', 'pine', 'willow',
'bicycle', 'bus', 'motorcycle', 'pickup truck', 'train',
'lawn-mower', 'rocket', 'streetcar', 'tank', 'tractor',
]

#### Модель

In [194]:
NN = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True).to(cuda0).train()

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


#### Обучение

In [195]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(NN.parameters(), lr = 0.001, momentum = 0.9)

In [196]:
n_epochs = 7

for epoch in range(n_epochs):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(cuda0), data[1].to(cuda0)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = NN(inputs)
        # print(outputs.device, inputs.device)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        # if i % 100 == 0:    # print every 2000 mini-batches
        #     print(f'[{epoch+1:3d}, {i+1:5d}] loss: {running_loss / 2000:.3f}')
        #           # (epoch + 1, i + 1, running_loss / 2000))
        #     running_loss = 0.0
    print(f'Epoch:{epoch+1:3d}, Running Loss:{running_loss:.3f}')

print('Finished Training')

Epoch:  1, Running Loss:1715.780
Epoch:  2, Running Loss:977.478
Epoch:  3, Running Loss:777.379
Epoch:  4, Running Loss:657.721
Epoch:  5, Running Loss:561.500
Epoch:  6, Running Loss:475.513
Epoch:  7, Running Loss:402.140
Finished Training


#### Проверка на всех классах

In [197]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
NN.eval()
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(cuda0), data[1].to(cuda0)
        # calculate outputs by running images through the network
        outputs = NN(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Точность ResNet18 для {total} изображений: {100*correct/total:.3f}')

Точность нейросети для 10000 изображений: 50.170


#### Проверка на каждом классе

In [198]:
correct_pred = {classname: 0 for classname in classes2}
total_pred = {classname: 0 for classname in classes2}

# again no gradients needed
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(cuda0), data[1].to(cuda0)
        outputs = NN(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes2[label]] += 1
            total_pred[classes2[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Точность ResNet18 для класса \'{classname:7s}\': {accuracy:.3f}')

Точность ResNet18 для класса 'beaver ': 79.000
Точность ResNet18 для класса 'dolphin': 63.000
Точность ResNet18 для класса 'otter  ': 40.000
Точность ResNet18 для класса 'seal   ': 19.000
Точность ResNet18 для класса 'whale  ': 34.000
Точность ResNet18 для класса 'aquarium fish': 58.000
Точность ResNet18 для класса 'flatfish': 53.000
Точность ResNet18 для класса 'ray    ': 55.000
Точность ResNet18 для класса 'shark  ': 58.000
Точность ResNet18 для класса 'trout  ': 60.000
Точность ResNet18 для класса 'orchids': 44.000
Точность ResNet18 для класса 'poppies': 31.000
Точность ResNet18 для класса 'roses  ': 55.000
Точность ResNet18 для класса 'sunflowers': 29.000
Точность ResNet18 для класса 'tulips ': 43.000
Точность ResNet18 для класса 'bottles': 45.000
Точность ResNet18 для класса 'bowls  ': 57.000
Точность ResNet18 для класса 'cans   ': 55.000
Точность ResNet18 для класса 'cups   ': 49.000
Точность ResNet18 для класса 'plates ': 36.000
Точность ResNet18 для класса 'apples ': 70.000
Точ